In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from cvxopt import matrix, solvers
np.random.seed(21)

means = [[2, 2], [4, 1]]
cov = [[.3, .2], [.2, .3]]
N = 10
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X1[-1, :] = [2.7, 2]
X = np.concatenate((X0.T, X1.T), axis = 1)
y = np.concatenate((np.ones((1, N)), -1*np.ones((1, N))), axis = 1)

In [15]:
# Solution found by sklearn
C = 100
clf = SVC(kernel = 'linear', C = C)
clf.fit(X.T, y.reshape(-1)) 

w_sklearn = clf.coef_.reshape(-1, 1)
b_sklearn = clf.intercept_[0]
print(w_sklearn.T, b_sklearn)

[[-5.54202362  2.4156074 ]] 9.132415590204586


In [3]:
# Solution found by dual problem
from cvxopt import matrix, solvers
# build K
V = np.concatenate((X0.T, -X1.T), axis = 1)
K = matrix(V.T.dot(V))

p = matrix(-np.ones((2*N, 1)))
# build A, b, G, h 

# Gx <= h -> 0 <= all lambda_n <= C
G = matrix(np.vstack((-np.eye(2*N), np.eye(2*N))))
h = matrix(np.vstack((np.zeros((2*N, 1)), C*np.ones((2*N, 1)))))

# Ax = b -> y.T.lambda = 0
A = matrix(y.reshape((-1, 2*N))) 
b = matrix(np.zeros((1, 1))) 
solvers.options['show_progress'] = False
sol = solvers.qp(K, p, G, h, A, b)

l = np.array(sol['x'])
print('lambda =')
print(l)


lambda =
[[1.26997770e-08]
 [7.29907090e-09]
 [6.75263620e+00]
 [1.20067067e-08]
 [8.83482181e-09]
 [1.00135373e-08]
 [9.49241066e-09]
 [1.10095260e-08]
 [1.09448265e-08]
 [1.15277180e+01]
 [3.06483278e-09]
 [2.92217775e-09]
 [3.52341246e-09]
 [5.49363383e-09]
 [4.48478627e-09]
 [7.55953464e-09]
 [2.73325320e-09]
 [5.71296652e-09]
 [5.02756847e-09]
 [1.82803543e+01]]


In [4]:
S = np.where(l > 1e-5)[0] # support set 
S2 = np.where(l < .999*C)[0] 

M = [val for val in S if val in S2] # intersection of two lists

VS = V[:, S]
lS = l[S]
yM = y[:, M]
XM = X[:, M]

w_dual = VS.dot(lS).reshape(-1, 1)
b_dual = np.mean(yM.T - w_dual.T.dot(XM))
print(w_dual.T, b_dual) 

[[-5.54276837  2.41628387]] 9.132906850859772


In [12]:
# Solution found by problem without constraints
X0_bar = np.vstack((np.ones((1, N)), X0.T))
X1_bar = np.vstack((np.ones((1, N)), X1.T))
Z = np.hstack((X0_bar, -X1_bar))
lam = 1./C

def cost(w):
    u = w.T.dot(Z)
    L = np.sum(np.maximum(0, 1 - u)) + 0.5*lam*np.sum(w*w) - 0.5*lam*w[0]*w[0]
    return L

def grad(w):
    u = w.T.dot(Z)
    H = np.where(u < 1)[1]
    ZS = Z[:, H]
    g = -np.sum(ZS, axis = 1, keepdims = True) + lam*w
    g[0] -= lam*w[0]
    return g

def num_grad(w):
    g = np.zeros_like(w)
    eps = 1e-6
    for i in range(len(w)):
        wp = w.copy()
        wm = w.copy()
        wp[i] += eps 
        wm[i] -= eps 
        g[i] = (cost(wp) - cost(wm))/(2*eps)
    return g 

w0 = np.random.randn(X0_bar.shape[0], 1) 
g1 = grad(w0)
g2 = num_grad(w0)
diff = np.linalg.norm(g1 - g2)
print('Gradient different:', diff)

Gradient different: 6.888024053287044e-09


In [14]:
def grad_descent(w0, eta):
    w = w0
    it =0
    while it < 100000:
        it = it + 1
        g = grad(w)
        w -= eta*g
        if (it % 10000) == 1:
            print('Iter', it, ', cost =', cost(w))
        if np.linalg.norm(g) < 1e-5:
            break 
    return w

w0 = np.random.randn(X0_bar.shape[0], 1) 
w = grad_descent(w0, 0.001)
w_hinge = w[:-1].reshape(-1, 1)
b_hinge = w[-1]
print(w_hinge.T, b_hinge)

Iter 1 , cost = [19.59757581]
Iter 10001 , cost = [1.29523096]
Iter 20001 , cost = [0.96717892]
Iter 30001 , cost = [0.66141401]
Iter 40001 , cost = [0.38027089]
Iter 50001 , cost = [0.20107483]
Iter 60001 , cost = [0.20096532]
Iter 70001 , cost = [0.19338311]
Iter 80001 , cost = [0.18341559]
Iter 90001 , cost = [0.18334596]
[[ 9.14466964 -5.54931207]] [2.41826607]
